In [1]:
from solartwins import *
from scipy import optimize
from tqdm import tqdm

In [2]:
t

star_name,age,age_err,Fe,Fe_err,O,O_err,Na,Na_err,Mg,Mg_err,Al,Al_err,Si,Si_err,S,S_err,Ca,Ca_err,V,V_err,Mn,Mn_err,Co,Co_err,Ni,Ni_err,Cu,Cu_err,Zn,Zn_err,Sr,Sr_err,Y,Y_err,Zr,Zr_err,Ba,Ba_err,La,La_err,Ce,Ce_err,Pr,Pr_err,Nd,Nd_err,Sm,Sm_err,Eu,Eu_err,Gd,Gd_err,Dy,Dy_err,C,C_err,Ti,Ti_err,Sc,Sc_err,Cr,Cr_err
bytes9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HIP10175,3.2,0.4,0.0,0.002,0.024,0.009,-0.051000000000000004,0.022,-0.006000000000000002,0.007,-0.034,0.008,-0.009999999999999998,0.003,0.0020000000000000018,0.012,0.023,0.005,-0.0019999999999999983,0.004,-0.029,0.005,-0.028,0.004,-0.04100000000000001,0.003,-0.05600000000000001,0.013,-0.048,0.006,0.081,0.005,0.066,0.007,0.087,0.012,0.11599999999999999,0.005,0.118,0.029,0.104,0.016,0.16999999999999998,0.008,0.127,0.01,0.057,0.013,0.089,0.005,0.08,0.005,0.065,0.005,-0.05176923076923076,0.013000000000000001,0.012,0.01,-0.019000000000000003,0.020999999999999998,0.007750000000000001,0.012
HIP101905,1.2,0.3,0.0,0.004,-0.06899999999999999,0.014,-0.093,0.012,-0.025999999999999995,0.008,-0.033999999999999996,0.002,-0.024999999999999994,0.004,-0.104,0.027,0.024000000000000007,0.005,0.0050000000000000044,0.009,-0.04599999999999999,0.006,-0.03799999999999999,0.005,-0.03799999999999999,0.005,-0.08499999999999999,0.031,-0.09699999999999999,0.016,0.097,0.006,0.08100000000000002,0.011,0.08499999999999999,0.01,0.14,0.01,0.10200000000000001,0.031,0.119,0.012,0.094,0.014,0.131,0.01,0.05399999999999999,0.021,0.07600000000000001,0.014,0.049000000000000016,0.006,0.066,0.066,-0.11374193548387096,0.031,0.009076923076923078,0.013000000000000001,-0.008058823529411757,0.017,-0.004749999999999994,0.016
HIP102040,2.4,0.4,0.0,0.003,0.022,0.006,-0.071,0.008,0.0020000000000000018,0.003,-0.021000000000000005,0.008,-0.0040000000000000036,0.005,-0.06499999999999999,0.014,0.034,0.005,0.0,0.006,-0.06299999999999999,0.005,-0.012999999999999998,0.007,-0.03799999999999999,0.004,-0.06599999999999999,0.034,-0.077,0.02,0.062,0.005,0.054000000000000006,0.007,0.081,0.011,0.128,0.012,0.118,0.018,0.14900000000000002,0.016,0.158,0.01,0.165,0.011,0.113,0.023,0.14600000000000002,0.017,0.094,0.006,0.108,0.028,-0.019789473684210527,0.056999999999999995,0.029545454545454552,0.011,0.0178,0.034999999999999996,-0.007249999999999993,0.016
HIP102152,8.6,0.3,0.0,0.003,0.051000000000000004,0.01,-0.027999999999999997,0.014,0.031,0.01,0.026000000000000002,0.004,0.003000000000000001,0.003,-0.012,0.022,0.006,0.005,-0.009000000000000001,0.006,-0.026999999999999996,0.005,-0.018000000000000002,0.008,-0.017,0.004,0.002,0.004,0.004,0.011,-0.11,0.005,-0.104,0.007,-0.094,0.012,-0.049,0.009,-0.062,0.019,-0.023,0.013,0.0,0.017,0.0010000000000000009,0.009,0.015,0.01,0.023,0.019,0.028,0.005,-0.031,0.027,0.001285714285714285,0.014,0.0035454545454545456,0.011,-0.00975,0.016,-0.002230769230769232,0.013
HIP10303,5.8,0.4,0.0,0.003,-0.022999999999999993,0.007,-0.0020000000000000018,0.012,0.017,0.005,0.033000000000000015,0.004,0.0040000000000000036,0.002,-0.023999999999999994,0.014,0.0050000000000000044,0.003,0.015,0.006,0.019000000000000003,0.004,0.0,0.005,0.020000000000000004,0.004,0.013999999999999999,0.021,0.0040000000000000036,0.02,0.032000000000000015,0.005,0.019000000000000003,0.006,0.0040000000000000036,0.014,-0.01999999999999999,0.008,-0.031,0.006,-0.027999999999999997,0.015,0.008000000000000007,0.012,-0.013999999999999999,0.007,-0.010999999999999996,0.016,0.01100000000000001,0.013,0.030000000000000013,0.005,-0.0049999999999999906,0.021,-0.033571428571428565,0.007,0.009727272727272737,0.011,0.005250

In [3]:
def delta_minimized(table, element):
    x0 = (1, 0, -1)
    deltatemp = []
    deltanotemp = []
    
    ages = table['age']
    age_error = table['age_err']
    abundance_temp = table[element]
    abundance_error = table[element + '_err']
    abundance_notemp = residuals(ages, abundance_temp, abundance_error)
    
    #BEFORE REMOVING TEMP TRENDS
    delt_temp = scipy.optimize.minimize(twodnlnL, x0, args = (ages, abundance_temp, age_error, abundance_error))
    tvalue = delt_temp['x'][2]
    deltatemp.append(tvalue)

    #AFTER REMOVING TEMP
    delt_notemp = scipy.optimize.minimize(twodnlnL, x0, args = (ages, abundance_notemp, age_error, abundance_error))
    value = delt_notemp['x'][2]
    deltanotemp.append(value)
        
    return deltatemp , deltanotemp

star_elements = []
for n in t.colnames:
    if len(n) < 3 :
        star_elements.append(n) #list of elements in table
elements = np.array(star_elements)
deltemp=[]
delnotemp =[]

for y in tqdm(elements):
    deltemp.append(delta_minimized(t,y)[0])
    delnotemp.append(delta_minimized(t,y)[1])

delttable = Table([elements, deltemp, delnotemp], names=('Elements', 'With Temp', 'Without Temp')) 
delttable

In [4]:
def jackknife_delta(x,y,erx, ery):
    N=100
    l=list(np.copy(x))
    k=list(np.copy(y))
    s=list(np.copy(erx))
    t=list(np.copy(ery))
    jacktemp= []
    jacknotemp= [] 
    h=0
    
    #leaving out one point from data set and calculating delta for each instance
    while h<N:
        w = random.randint(0, (len(x)-1))
        del l[w]
        del k[w]
        del s[w] 
        del t[w] #removing one data set from lists 
    
        #BEFORE REMOVING TEMP TRENDS
        x0 = (1, 0, -1)
        delt_temp = scipy.optimize.minimize(twodnlnL, x0, args = (l,k,s,t))
        tvalue = delt_temp['x'][2]
        jacktemp.append(tvalue)
        
        #AFTER REMOVING TEMP
        abundance_notemp = residuals(l, k, t)
        delt_notemp = scipy.optimize.minimize(twodnlnL, x0, args = (l, abundance_notemp, s, t))
        value = delt_notemp['x'][2]
        jacknotemp.append(value)
            
        l=list(np.copy(x)) #adding value back in for next round 
        k=list(np.copy(y)) 
        s=list(np.copy(erx))
        t=list(np.copy(ery))
        h=h+1 
        
    return jacktemp, jacknotemp

In [5]:
def age_abund_plot(table):
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 15
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size

    star_elements = []
    for n in t.colnames:
        if len(n) < 3 :
            star_elements.append(n) #list of elements in table
    elements = np.array(star_elements)
    deltemp=[]

    for y, ele in tqdm(enumerate(elements)):
        deltemp.append(delta_minimized(t, ele)[0])
    
        x0 = (.01, .03, .07) #initial guess
        restemp = scipy.optimize.minimize(twodnlnL, x0, args = (t['age'], t[ele], t['age_err'], t[ele + '_err']))
    
        plt.ioff()
        fig, ax = plt.subplots()
    
        ax.scatter(t['age'], t[ele], c='rebeccapurple') 
        ax.set_xlabel('Age',fontsize='xx-large')
        ax.set_ylabel(ele + '/Fe', fontsize='xx-large')
        ax.set_title(ele + ' Abundance vs. Stellar Age', fontsize='xx-large')

        #line of best fit
        mb = find_m_b(t['age'], t[ele], t[ele + '_err'])
        for i, txt in enumerate (t[ele]):
            plot_xs = np.arange(0, 9, .01)
            ax.plot(plot_xs, mb[0] * plot_xs + (mb[1]), color = 'olivedrab', linewidth=1)
    
        #point labels
        #for i, txt in enumerate(t['star_name']): 
                #ax.annotate(txt, xy=(t['age'][i], t[ele][i]), xytext=(-5,5), 
                    #textcoords='offset points', ha='center', va='bottom')
            
        mbtemp= restemp['x']
        plot_xs = np.arange(0, 9, .1)
        ax.plot(plot_xs, mbtemp[0] * plot_xs + (mbtemp[1]), color = 'black', linewidth=1)
        #ax.text(2,.15, deltemp[y] ,horizontalalignment='right', fontsize=12)
    
        fig.savefig(ele +'_age.png')
        plt.close(fig)

In [6]:
def age_abund_plot_no_temp(table):
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 15
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size

    star_elements = []
    for n in t.colnames:
        if len(n) < 3 :
            star_elements.append(n) #list of elements in table
    elements = np.array(star_elements)
    delnotemp=[]

    for d, nam in tqdm(enumerate(elements)):
        delnotemp.append(delta_minimized(t, nam)[1])
        abundance_notemp = residuals(t['age'], t[nam], t[nam + '_err'])

        x0 = (.01, .03, .07) #initial guess
        resnotemp = scipy.optimize.minimize(twodnlnL, x0, args = (t['age'], abundance_notemp, t['age_err'], t[nam + '_err']))
    
        plt.ioff()
        fig, ax = plt.subplots()
    
        ax.scatter(t['age'], abundance_notemp, c='forestgreen') 
        ax.set_xlabel('Age',fontsize='xx-large')
        ax.set_ylabel(nam + '/Fe', fontsize='xx-large')
        ax.set_title(nam + ' Abundance vs. Stellar Age', fontsize='xx-large')

        #line of best fit
        mb = find_m_b(t['age'], abundance_notemp, t[nam + '_err'])
        for i, txt in enumerate (t[nam]):
            plot_xs = np.arange(0, 9, .01)
            ax.plot(plot_xs, mb[0] * plot_xs + (mb[1]), color = 'palevioletred', linewidth=1)
    
        #point labels
        #for i, txt in enumerate(t['star_name']): 
                #ax.annotate(txt, xy=(t['age'][i], t[ele][i]), xytext=(-5,5), 
                    #textcoords='offset points', ha='center', va='bottom')
            
        mbtemp= resnotemp['x']
        plot_xs = np.arange(0, 9, .1)
        ax.plot(plot_xs, mbtemp[0] * plot_xs + (mbtemp[1]), color = 'black', linewidth=1)
        #ax.text(2,.15, deltemp[y] ,horizontalalignment='right', fontsize=12)
    
        fig.savefig(nam +'_age_no_temp.png')
        plt.close(fig)

In [7]:
stardeltas = []
stardeltas_notemp =[]
for r, star in enumerate(t['star_name']):
    tbl = t[r] #inputted star's row 
    x0 = (.01, .03, .07)
    
    star_elements = []
    for n in tbl.colnames:
        if len(n) < 3 :
            star_elements.append(n) #list of elements in table
    elements = np.array(star_elements)
    
    con_temp = []
    for n in elements: 
        con_temp.append(tc_map[n])
    temps = np.copy(con_temp)
    random.shuffle(temps)
    
    star_abundance = []
    abundance_error = []
    for n in elements:
        star_abundance.append(tbl[n])
        abundance_error.append(tbl[n + '_err'])
        
    delt_temp = scipy.optimize.minimize(nlnL, x0, args = (con_temp, star_abundance, abundance_error))
    deltavlue = delt_temp['x'][2]
    stardeltas.append(deltavlue)
    

    delt_notemp = scipy.optimize.minimize(nlnL, x0, args = (temps, star_abundance, abundance_error))
    deltanotempvlue = delt_notemp['x'][2]
    stardeltas_notemp.append(deltanotempvlue)

delttable = Table([stardeltas, stardeltas_notemp], names=('With Temp', 'Without Temp')) 
delttable

/Users/ilanadoran/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


KeyboardInterrupt: 

In [ ]:
age_abund_plot(t)

In [ ]:
age_abund_plot_no_temp(t)